# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import required libraries
import re
import numpy as np
import pandas as pd
import nltk
import sqlite3
nltk.download(['punkt', 'wordnet'])
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database

engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine) #df = pd.read_sql_table('DisasterResponse', 'sqlite:///DisasterResponse.db') 
df = df.replace(to_replace='None', value=np.nan)


In [3]:
# It was observed that some rows contain "#NAME?". Those rows will be dropped from the dataset 
# as they are not useful and can lead our model to crash

df=df[df["message"]!='#NAME?']

In [4]:
# Obtain X and Y from the data

X = df["message"] 
Y = df.loc[:,"related":"direct_report"]

df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    '''
    INPUT:
    df: a text (string format)
      
    
    OUTPUT:
    clear tokens: a list of strings, obtained as a result of the following operations on the input text:
            - Everything but letters (uppercase und lowercase) and numbers will be removed
            - Text will be divided into separate elements, or "tokens"
            - Stop words corresponding to the English language will be removed
            - Tokens will be lemmatized, i.e. tokens will be converted into "root words",
                based on WordNetLemmatizer
            - Tokens will be lemmatized, i.e. tokens will be converted into "root words"
            - Tokens will be normalized
   
    '''
    
    # normalize text
    text = re.sub(r"[^a-zA-Z0-9]"," ",text)
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # remove tokens corresponding to stop words
    tokens = [word for word in tokens if not word in stopwords.words("english")]
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens 

#    pass

In [6]:
# test out function
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')


Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
# Pipeline will be built with the following features:
# - CountVectorizer as vectorizer, with tokenize as tokenizer
# - tf-idf transformation
# - MultiOutputClassifier (subtype KNeighborsClassifier) as classifier

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.6)
    
# Pipeline will be traied with train data (X_train and y_train)
pipeline.fit(X_train, y_train)   

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
# Predicted values for the X_test and X_train will be obtained
y_test_pred = pipeline.predict(X_test)
y_train_pred = pipeline.predict(X_train)

In [10]:
# y_test_pred and y_train_pred are obtaines as numpy arrays
# for further operations, we need to convert them into dataframe
# therefore, y_test_pred_df and y_train_pred_df are introduced:

y_headers = y_test.columns

y_test_pred_df = pd.DataFrame(y_test_pred, columns = y_headers)

y_train_pred_df = pd.DataFrame(y_train_pred, columns = y_headers)

In [11]:
# Model will be evaluated through the test values
# by using the Classification_Report function

for col in y_test:
    test_score = classification_report(y_test[col],y_test_pred_df[col],)
    print(col)
    print(test_score)


related
             precision    recall  f1-score   support

          0       0.44      0.60      0.51      3586
          1       0.86      0.77      0.82     12010
          2       0.63      0.15      0.25       110

avg / total       0.76      0.73      0.74     15706

request
             precision    recall  f1-score   support

          0       0.88      0.97      0.92     12974
          1       0.71      0.38      0.50      2732

avg / total       0.85      0.87      0.85     15706

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     15646
          1       0.00      0.00      0.00        60

avg / total       0.99      1.00      0.99     15706

aid_related
             precision    recall  f1-score   support

          0       0.67      0.87      0.76      9134
          1       0.69      0.40      0.51      6572

avg / total       0.68      0.67      0.65     15706

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
# Model will be evaluated through the train values
# by using the Classification_Report function

for col in y_train:
    train_score = classification_report(y_train[col],y_train_pred_df[col],)
    print(col)
    print(train_score)

related
             precision    recall  f1-score   support

          0       0.63      0.79      0.70      2528
          1       0.92      0.85      0.88      7864
          2       0.76      0.37      0.50        78

avg / total       0.85      0.83      0.84     10470

request
             precision    recall  f1-score   support

          0       0.91      0.99      0.94      8738
          1       0.87      0.49      0.63      1732

avg / total       0.90      0.90      0.89     10470

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00     10413
          1       0.50      0.02      0.03        57

avg / total       0.99      0.99      0.99     10470

aid_related
             precision    recall  f1-score   support

          0       0.75      0.95      0.84      6202
          1       0.88      0.54      0.67      4268

avg / total       0.80      0.78      0.77     10470

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])

parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
         'vect__max_df': (0.1, 0.5, 1),
        'vect__max_features': (None, 1000),
        'clf__estimator__n_neighbors': [2, 5]
              }

cv = GridSearchCV(pipeline, parameters)
cv.fit(X_train, y_train)

print("Best parameters = ")
best_parameters = cv.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

Best parameters = 
clf__estimator__n_neighbors: 2
vect__max_df: 0.5
vect__max_features: None
vect__ngram_range: (1, 2)


In [14]:
# run this piece of code in order to know
# which hyperparameters are eligible for GridSearcvCV in our pipeline

for param in cv.get_params().keys():
    print(param)

cv
error_score
estimator__memory
estimator__steps
estimator__vect
estimator__tfidf
estimator__clf
estimator__vect__analyzer
estimator__vect__binary
estimator__vect__decode_error
estimator__vect__dtype
estimator__vect__encoding
estimator__vect__input
estimator__vect__lowercase
estimator__vect__max_df
estimator__vect__max_features
estimator__vect__min_df
estimator__vect__ngram_range
estimator__vect__preprocessor
estimator__vect__stop_words
estimator__vect__strip_accents
estimator__vect__token_pattern
estimator__vect__tokenizer
estimator__vect__vocabulary
estimator__tfidf__norm
estimator__tfidf__smooth_idf
estimator__tfidf__sublinear_tf
estimator__tfidf__use_idf
estimator__clf__estimator__algorithm
estimator__clf__estimator__leaf_size
estimator__clf__estimator__metric
estimator__clf__estimator__metric_params
estimator__clf__estimator__n_jobs
estimator__clf__estimator__n_neighbors
estimator__clf__estimator__p
estimator__clf__estimator__weights
estimator__clf__estimator
estimator__clf__n_jo

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
# Predicted values for the X_test and X_train will be obtained by inserting
# in the pipeline the best values obtained thanks to the GridSearchCV

y_test_pred = cv.best_estimator_.predict(X_test)
y_train_pred = cv.best_estimator_.predict(X_train)

In [16]:
# y_test_pred and y_train_pred are obtaines as numpy arrays
# for further operations, we need to convert them into dataframe
# therefore, y_test_pred_df and y_train_pred_df are introduced:

y_headers = y_test.columns

y_test_pred_df = pd.DataFrame(y_test_pred, columns = y_headers)

y_train_pred_df = pd.DataFrame(y_train_pred, columns = y_headers)

In [17]:
# Model will be evaluated through the test values
# by using the Classification_Report function

for col in y_test:
    test_score = classification_report(y_test[col],y_test_pred_df[col],)
    print(col)
    print(test_score)

related
             precision    recall  f1-score   support

          0       0.43      0.52      0.47      3586
          1       0.84      0.79      0.82     12010
          2       0.36      0.24      0.29       110

avg / total       0.74      0.73      0.73     15706

request
             precision    recall  f1-score   support

          0       0.83      1.00      0.91     12974
          1       0.78      0.01      0.02      2732

avg / total       0.82      0.83      0.75     15706

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     15646
          1       0.00      0.00      0.00        60

avg / total       0.99      1.00      0.99     15706

aid_related
             precision    recall  f1-score   support

          0       0.58      1.00      0.74      9134
          1       0.63      0.01      0.02      6572

avg / total       0.60      0.58      0.43     15706

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [18]:
# Model will be evaluated through the train values
# by using the Classification_Report function

for col in y_train:
    train_score = classification_report(y_train[col],y_train_pred_df[col],)
    print(col)
    print(train_score)

related
             precision    recall  f1-score   support

          0       0.97      1.00      0.99      2528
          1       1.00      0.99      1.00      7864
          2       1.00      0.95      0.97        78

avg / total       0.99      0.99      0.99     10470

request
             precision    recall  f1-score   support

          0       0.84      1.00      0.91      8738
          1       1.00      0.03      0.06      1732

avg / total       0.87      0.84      0.77     10470

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00     10413
          1       0.00      0.00      0.00        57

avg / total       0.99      0.99      0.99     10470

aid_related
             precision    recall  f1-score   support

          0       0.60      1.00      0.75      6202
          1       1.00      0.03      0.05      4268

avg / total       0.76      0.60      0.47     10470

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
# New properties can be added to the model
# For example, we can try with "RandomForestClassifier", instead of "KNeighborsClassifier", as a classifier


new_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier ()))
    ])


# train the new model
new_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 9. Export your model as a pickle file

In [20]:
# Save Pipeline model (with parameters already optimized thanks ti GridSearchCV)
# as a pickle file

# pkl_filename = '{}'.format(model_filepath)
# with open(pkl_filename, 'wb') as file:
pickle.dump(cv, open('final_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.